In [1]:
import os, sys

def list_subdirectories(path):
    subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return subdirs

def list_subfiles(path):
    subfilesPath = files = [os.path.join(root, f) for root, _, files in os.walk(path) for f in files]
    return subfilesPath

# path = "/mnt/c/Users/hoang/FileCSV_DACN_2025/opt/Malware-Project/BigDataset/IoTScenarios"
path = "C:/Users/hoang/FileCSV_DACN_2025/opt/Malware-Project/BigDataset/IoTScenarios"
subdirectories = list_subdirectories(path)

filesPath = []
print("Các thư mục con:")
for subdir in subdirectories:
    print(subdir)
    for f in list_subfiles(path+"/"+subdir):
        filesPath.append(f)
    
# print(filesPath)

import csv
import re
import dask.dataframe as dd
import pandas as pd

input_file = filesPath[0]
# output_file = '/mnt/c/Users/hoang/FileCSV_DACN_2025/opt/log_1.csv'
# output_file2 = '/mnt/c/Users/hoang/FileCSV_DACN_2025/opt/log_2.csv'
output_file = "C:/Users/hoang/FileCSV_DACN_2025/opt/log_1.csv"
output_file2 = "C:/Users/hoang/FileCSV_DACN_2025/opt/log_2.csv"

fields = []
with open(input_file, "r") as f:
    for line in f:
        line=line.strip()
        if line.startswith("#fields"):
            fields = re.split(r'[\t ]+',line)[1:]  # Bỏ '#fields'
            break

# fields[-1] = fields[-1].replace('\n', '')
print(len(fields))

header_Existence = False
for indexFile, filePath in enumerate(filesPath):
    df = dd.read_csv(
        filePath,
        sep=r'[\t ]+',
        comment='#',
        names=fields,
        dtype=str,
        engine='python',
        blocksize="64MB",
        on_bad_lines='skip'
    )
    
    print(f"file Index: {indexFile}")
    #, os.path.getsize(filePath)
    # print(df.head())
    # print(df.shape[0].compute())
    df.to_csv(output_file, mode='a', header=not header_Existence, index=False, single_file=True)
    header_Existence = True

Các thư mục con:
CTU-Honeypot-Capture-4-1
CTU-Honeypot-Capture-5-1
CTU-Honeypot-Capture-7-1
CTU-IoT-Malware-Capture-1-1
CTU-IoT-Malware-Capture-17-1
CTU-IoT-Malware-Capture-20-1
CTU-IoT-Malware-Capture-21-1
CTU-IoT-Malware-Capture-3-1
CTU-IoT-Malware-Capture-33-1
CTU-IoT-Malware-Capture-34-1
CTU-IoT-Malware-Capture-35-1
CTU-IoT-Malware-Capture-36-1
CTU-IoT-Malware-Capture-39-1
CTU-IoT-Malware-Capture-42-1
CTU-IoT-Malware-Capture-43-1
CTU-IoT-Malware-Capture-44-1
CTU-IoT-Malware-Capture-48-1
CTU-IoT-Malware-Capture-49-1
CTU-IoT-Malware-Capture-52-1
CTU-IoT-Malware-Capture-60-1
CTU-IoT-Malware-Capture-7-1
CTU-IoT-Malware-Capture-8-1
CTU-IoT-Malware-Capture-9-1
23
file Index: 0
file Index: 1
file Index: 2
file Index: 3
file Index: 4
file Index: 5
file Index: 6
file Index: 7
file Index: 8
file Index: 9
file Index: 10
file Index: 11
file Index: 12
file Index: 13
file Index: 14


MemoryError: Unable to allocate 81.0 MiB for an array with shape (461450, 23) and data type object

# Data cleaning, Filtering, Normalization, ...

In [5]:

df = dd.read_csv(output_file, dtype=str)
print(df.shape)
# Step 4: Data cleaning
non_empty_columns = df.columns[df.isnull().sum().compute() < len(df)]
df = df[non_empty_columns]
df = df.drop_duplicates()             # Drop duplicate rows
df = df.fillna("0")                   # Replace missing/null with "0"

label_mapping= {"benign":0, "Malicious":1, "Benign":0, "malicious": 1}
df['label'] = df['label'].map(label_mapping).fillna(-1).astype(int)

# Step 6: Save cleaned data for ML
df.to_csv(output_file2, single_file=True, index=False)

(<dask_expr.expr.Scalar: expr=ArrowStringConversion(frame=FromMapProjectable(c028377)).size() // 23, dtype=int64>, 23)


c:\Users\hoang\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\dask_expr\_collection.py:4208: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('label', 'float64'))

  warnings.warn(meta_warning(meta))


['C:\\Users\\hoang\\FileCSV_DACN_2025\\opt\\log_2.csv']